In [7]:
import os
import torch
import time
from PIL import Image
import numpy as np
from torchvision import transforms
import onnx
import onnxruntime

In [8]:
#checking the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [9]:
#importing the models from the compressai zoo
from compressai.zoo import models
#models avaialble are 
model_names=models.keys()
print("models are:",list(model_names))

quality=4
metric="ms-ssim" #"mse"
model_name="bmshj2018-factorized"
#model_name="mbt2018-mean"
#mode_name="mbt2018-mean"

net = models[model_name](quality=quality,metric=metric, pretrained=True).eval().to(device)
print("Model encoder architecture",model_name)

print(f'Encoder Parameters: {sum(p.numel() for p in net.parameters())}')

models are: ['bmshj2018-factorized', 'bmshj2018-factorized-relu', 'bmshj2018-hyperprior', 'mbt2018-mean', 'mbt2018', 'cheng2020-anchor', 'cheng2020-attn', 'bmshj2018-hyperprior-vbr', 'mbt2018-mean-vbr', 'mbt2018-vbr', 'hrtzxf2022-pcc-rec', 'sfu2023-pcc-rec-pointnet', 'sfu2024-pcc-rec-pointnet2-ssg', 'ssf2020']
Model encoder architecture bmshj2018-factorized
Encoder Parameters: 2998147


In [30]:
#trying indivisually for each model configuration
dl_models={} 

model_name="bmshj2018-factorized"

export_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\pytorch_models"

for quality in [4,6,8]:
    for metric in ["mse","ms-ssim"]:
        net=models[model_name](quality=quality,metric=metric,pretrained=True).eval().to(device)
        parameters=sum(p.numel() for p in net.parameters())
        model_name_final=model_name+"_"+str(quality)+"_"+metric
        dl_models[model_name_final]=[net,[model_name,quality,metric,parameters]]
        print("model_name:",model_name,"quality:",quality,"metric:",metric,"param:",parameters)
        model_export_name=model_name_final
        print()
        print("Now exporting the model")
        model_export_path=os.path.join(export_path,model_export_name)

        #input for the model 

        
        torch.save(net,model_export_path)
        print("model is exported")
        time.sleep(1)    
            
        #except:
        #    print("FAILED model_name:",model_name,"quality:",quality,"metric:",metric,"param:",parameters)
        #time.sleep(5)
            

model_name: bmshj2018-factorized quality: 4 metric: mse param: 2998147

Now exporting the model
model is exported
model_name: bmshj2018-factorized quality: 4 metric: ms-ssim param: 2998147

Now exporting the model
model is exported
model_name: bmshj2018-factorized quality: 6 metric: mse param: 7030531

Now exporting the model
model is exported
model_name: bmshj2018-factorized quality: 6 metric: ms-ssim param: 7030531

Now exporting the model
model is exported
model_name: bmshj2018-factorized quality: 8 metric: mse param: 7030531

Now exporting the model
model is exported
model_name: bmshj2018-factorized quality: 8 metric: ms-ssim param: 7030531

Now exporting the model
model is exported


In [ ]:
#compress ai models dictionary

#dl_models={} #contains the model name and the parameter and the quality 
#for model_name in model_names:
#    for quality in [4,6,8]:
#        for metric in ["mse","ms-ssim"]:
#            try:
#                net=models[model_name](quality=quality,metric=metric,pretrained=True).eval().to(device)
#                parameters=sum(p.numel() for p in net.parameters())
#                model_name_final=mode_name+"_"+str(quality)+"_"+metric
#                dl_models[model_name_final]=[net,[model_name,quality,metric,parameters]]
#                print("model_name:",model_name,"quality:",quality,"metric:",metric,"param:",parameters)
#            except:
#                print("FAILED model_name:",model_name,"quality:",quality,"metric:",metric,"param:",parameters)
#            time.sleep(5)

#print("models we received are:",dl_models.keys())

In [ ]:
#writing the function to export the models to the disc 

In [5]:
#writing the functions to export the models to the ONNX versions

In [58]:
#Dataset handling
dataset_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\Dataset_50"
export_dataset_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\CompressAI Models\modified_dataset"


def image_handling(dataset_path):
    lst=os.listdir(dataset_path)
    images=[]
    for image in lst:
        if image.lower().endswith("jpg") or image.lower().endswith("jpeg") or image.lower().endswith("png") or image.lower().endswith("webp"):
            images.append(image)
    print("dataset has:",len(images),"images")

    return images


def preprocess_gray(image_path,image_shape):
    """
        image_shape is expected to be a tuple
        
        This function resizes the image and converts
        it to the grayscale and returns in the form of 
        numpy array
        """
    image=Image.open(image_path).convert("L")
    #resize the image
    image=image.resize(image_shape)
    
    #convert to numpy and normalize
    image=np.array(image)/255.0

    #print("image shape:",image.shape)
    
    #create an empty numpy array
    temp=np.zeros(image_shape)
    temp=np.expand_dims(temp,axis=0)

    #now make the array of size 3,shape
    img_blank=np.repeat(temp,3,axis=0)
    #img_blank=np.zeros((3,1232,1640))

    #assign the gray image to the blank array
    img_blank[0]=image
    
    img_final=img_blank.astype(np.float32)
    #img_final=np.expand_dims(img_blank,axis=0).astype(np.float32)
    #print("shape of the final image:",img_final.shape,"type:",type(img_final))
   
    return img_final


def save_compressed_image(output, save_path):
    """
    Save the compressed output image from the model inference.
    """
    # Post-process the output (if needed)
    output = output.squeeze(0)  # Remove batch dimension
    output=output[0]
    print("dimension",output.shape)
    
    # Convert to HWC format (Height x Width x Channels)
    #output = np.transpose(output, (1, 2, 0))
    
    # Clip values to valid range (0, 1) and convert to 8-bit (0-255)
    output = np.clip(output, 0, 1) * 255.0
    output = output.astype(np.uint8)
    
    # Convert to image format and save
    output_image = Image.fromarray(output,mode="L")
    print(np.array(output_image).shape)
    #output_image=output_image.convert("L")
    output_image.save(save_path,format="JPEG",quality=90) 


def export_modified_dataset(dataset_path,export_dataset_path,image_shape,gray):

    """
        image_shape is the shape of the expected images
    the original images will be resized to this shape

    gray is boolean, if True the dataset will be turned into gray
    else not
    
    """
    if gray==True:
        export_dataset_path=os.path.join(export_dataset_path,"gray")
    else:
        export_dataset_path=os.path.join(export_dataset_path,"color")
        
    if os.path.exists(export_dataset_path):
        print("export directory already exists")
    else:
        #make the path
        os.makedirs(export_dataset_path)
        print("export directory created")

    #getting the dataset
    images=image_handling(dataset_path)

    print("Using the image shape as:",image_shape)

    
    if gray==True:
        for image in images:
            image_path=os.path.join(dataset_path,image)
            img=preprocess_gray(image_path,image_shape)
            save_path=os.path.join(export_dataset_path,image.split('.')[0]+"_gray.png")#exporting the dataset as the PNG
            img=img[0]
            #img=img.transpose()
            #print("shape:",img.shape,"type :",type(img))
            img=np.clip(img,0,1)*255.0
            img=img.astype(np.uint8)
            img=Image.fromarray(img,mode="L")
            img.save(save_path,format="PNG") #quality not defined using the highest by default
    else:
        #we just resize the images based on the final size
        for image in images:
            image_path=os.path.join(dataset_path,image)
            #open the image and resize the image
            img=Image.open(image_path)
            #resize the image
            img=img.resize(image_shape)
            #convert to numpy and normalize
            img=np.array(img)/255.0
            img=np.clip(img,0,1)*255.0
            img=img.astype(np.uint8)
            save_path=os.path.join(export_dataset_path,image.split('.')[0]+"_resize.png")#exporting the dataset as the PNG
            img=Image.fromarray(img)
            img.save(save_path,format="PNG") #quality not defined using the highest by default
            
    print("The dataset modification is completed")
    return export_dataset_path
        
    
        


image_shape=(1280,1280)
gray=False

#final dataset path is the function output which will be used finally based on the gray option
export_dataset_path=export_modified_dataset(dataset_path,export_dataset_path,image_shape,gray)


export directory created
dataset has: 50 images
Using the image shape as: (1280, 1280)


In [7]:
#get write the functions to get the quality of the output from the compression in terms of PSNR SSIM and LPIPS
